# Bibliotecas

In [1]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [2]:
df_americanas = pd.read_csv(f"Dados/produtos_americanas.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/produtos_amazon.csv", dtype = {'ean': str})

In [3]:
df_amazon["ean"].value_counts()

0713477350430    57
9095247871972    51
7908414420304     2
7319625661149     1
7465214678507     1
                 ..
0708570152586     1
8263430934112     1
6973553520094     1
3041412361256     1
6934177770012     1
Name: ean, Length: 1017, dtype: int64

## Retirando títulos com palavras indesejadas

In [4]:
bad_words = ["acessório", "acessorio", "adaptador", "alargadores", "anel", "arduino", "atomiser", "avatar",
             "aparelho de som", "automotivo", "axiom",
             "bandeja", "biometria", "bike", "boroscópio", "box", 
             "cable", "cabo", "caixa", "campainha", "calculator", "calculadora", "capa", "capinha", "card",
             "case", "caso", "carregador", "carro"
             "chassi", "chaveiro", "chave seletora", "clip",
             "coletor", "collar", "conjunto", "comoda", "cômoda", "cordão", "cortador", "cosplay",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos", "gaveta",
             "handmade", "hemobblo", "hemobllo", "hd externo", "jbl", "junta", "kit",
             "lente", "lixa", "lurrose",
             "m-disc", "mágico", "magico", "mesa", "microsd", "monitor", "montagem", "moochy",
             "netcna", "óculos", "oculos", "ornament",
             "papel", "parede", "part", "partes", "pedal", "pen drive", "pendrive", "pneu", "pneus", "power bank",
             "prato", "prettyzoom", "pretyzoom", "proteção", "protetor", "pulseira",
             "raspberry", "radiador", "rank de memória", "realidade virtual", "refrigerador", "regeneradora",
             "relogio", "relógio", "remoção", "reparo", "reposição", "reposiçao", "resfriador", "retrovisor", "roda",
             "ronyme", "rv", "scanner", "sensor", "smart tv", "soquete", "stick", "streaming box", "suporte",
             "tablet", "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "ultra-sensível", "upgrade", "upgrades",
             "vador", "veado", "vedação", "verbatim", "vidro", "vr", "xilofone", "xylophone",
             "watch", "zawa", "zbox"
            ]

print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"\nFiltrado:\nAmazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (4333, 4)
Americanas: (1032, 4)

Filtrado:
Amazon: (1911, 5)
Americanas: (876, 5)


In [5]:
#df_amazon_ft['url'].loc[970]

In [6]:
#df_amazon_ft[350:].head(50)

## Encontrando EANs repetidos na mesma loja

Removendo produtos com o mesmo título

In [7]:
df_amazon_ft = df_amazon_ft.loc[~df_amazon_ft["titulo"].duplicated(keep = 'first')]
df_americanas_ft = df_americanas_ft.loc[~df_americanas_ft["titulo"].duplicated(keep = 'first')]

In [8]:
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (1738, 5)
Americanas: (815, 5)


Verificando Títulos de EANs duplicados

In [9]:
ama_vc_ean = df_amazon_ft[df_amazon_ft["ean"].notnull()]["ean"].value_counts()
ama_duplicados = ama_vc_ean[ama_vc_ean > 1].index[0]
df_amazon_ft[df_amazon_ft['ean'] == ama_duplicados]

,url,titulo,ean,descricao,titulo_cb
262,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 2gb + 64gb tela ...
2632,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 Dourado 64GB 2GB...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 dourado 64gb 2gb...


In [10]:
ame_vc_ean = df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()
ame_duplicados = ame_vc_ean[ame_vc_ean > 1].index
df_americanas_ft[df_americanas_ft["ean"].isin(ame_duplicados)].sort_values(by = ['ean']).head(1)

,descricao,ean,titulo,url,titulo_cb
459,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,https://www.americanas.com.br/produto/4431267188,redmi note 10 pro dual sim 128 gb 6 gb azul


In [11]:
df_americanas_ft['loja'] = "americanas"
df_amazon_ft['loja'] = "amazon"

In [12]:
frames = [df_americanas_ft, df_amazon_ft]
df_final = pd.concat(frames, ignore_index = True)

In [13]:
vc_ean = df_final[df_final["ean"].notnull()]["ean"].value_counts()
duplicados = vc_ean[vc_ean > 1].index
df_final_dup = df_final[df_final["ean"].isin(duplicados)].sort_values(by = ['ean'])

## Fazendo match

In [14]:
colunas = ["descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2"]

df_pares = pd.DataFrame(columns = colunas)
df_pares

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2


In [16]:
len(df_final_dup)

139

In [17]:
lista_ean = []

for i in range(len(df_final_dup)):
    
    if df_final_dup.iloc[i]['ean'] not in lista_ean:
        
        lista_ean.append(df_final_dup.iloc[i]['ean'])
        
        indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
        
        for j in range(len(indice)-1):
            
            for k in range(j + 1, len(indice)):
            
                df_pares = df_pares.append({'descricao_1' : df_final_dup.loc[indice[j]]['descricao'] , 'ean_1' : df_final_dup.loc[indice[j]]['ean'],
                                        'titulo_1' : df_final_dup.loc[indice[j]]['titulo'], 'url_1' : df_final_dup.loc[indice[j]]['url'],
                                        'titulo_cb_1' : df_final_dup.loc[indice[j]]['titulo_cb'], 'loja_1' : df_final_dup.loc[indice[j]]['loja'],
                                        
                                        'descricao_2' : df_final_dup.loc[indice[k]]['descricao'] , 'ean_2' : df_final_dup.loc[indice[k]]['ean'],
                                        'titulo_2' : df_final_dup.loc[indice[k]]['titulo'], 'url_2' : df_final_dup.loc[indice[k]]['url'] ,
                                        'titulo_cb_2' : df_final_dup.loc[indice[k]]['titulo_cb'], 'loja_2' : df_final_dup.loc[indice[k]]['loja']}
                                        , ignore_index=True)

In [18]:
#df_pares[df_pares['ean_1']=="6934177738180"][['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]
df_pares[['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]

,descricao_1,ean_1,titulo_1,descricao_2,ean_2,titulo_2,loja_1,loja_2,url_1,url_2
0,Todas as informações divulgadas são de respons...,0194252040485,"iPhone 12 Pro Apple (512GB) Prateado Tela 6,1""...",Sobre este item IPhone 12 pro 512GB D...,0194252040485,"Iphone 12 Pro Apple Prata, 512gb Desbloqueado ...",americanas,amazon,https://www.americanas.com.br/produto/2290962881,https://www.amazon.com.br/Iphone-Apple-Prata-5...
1,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 Gb Cinza 6 Gb,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,americanas,americanas,https://www.americanas.com.br/produto/4431295470,https://www.americanas.com.br/produto/4431267188
2,Desempenho superior com Snapdragon 860O que ve...,6934177738180,Poco X3 Pro Dual sim 256GB 8GB ram - Preto,Sistema operacional Nome do sistema operaciona...,6934177738180,Pocophone Poco X3 Pro Dual sim 256GB Preto 8GB...,americanas,americanas,https://www.americanas.com.br/produto/4540396296,https://www.americanas.com.br/produto/4511090415
3,Desempenho superior com Snapdragon 860O que ve...,6934177738180,Poco X3 Pro Dual sim 256GB 8GB ram - Preto,"Sobre este item ATENÇÃO: VERSÃO INDIANA, CO...",6934177738180,Smartphone Poco X3 PRO 256gb 8gb RAM – Phantom...,americanas,amazon,https://www.americanas.com.br/produto/4540396296,https://www.amazon.com.br/Smartphone-Poco-PRO-...
4,Sistema operacional Nome do sistema operaciona...,6934177738180,Pocophone Poco X3 Pro Dual sim 256GB Preto 8GB...,"Sobre este item ATENÇÃO: VERSÃO INDIANA, CO...",6934177738180,Smartphone Poco X3 PRO 256gb 8gb RAM – Phantom...,americanas,amazon,https://www.americanas.com.br/produto/4511090415,https://www.amazon.com.br/Smartphone-Poco-PRO-...
...,...,...,...,...,...,...,...,...,...,...
75,Multilaser G Max 2 com tela de 6.5 hd + u-notc...,7908414420304,Smartphone GMax 2 P9157 Tela 6.5 64GB 2GB ram ...,Sobre este item Câmera Tripla 8MP + VGA + V...,7908414420304,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,americanas,amazon,https://www.americanas.com.br/produto/4782017527,https://www.amazon.com.br/Smartphone-Multilase...
76,Multilaser G Max 2 com tela de 6.5 hd + u-notc...,7908414420304,Smartphone GMax 2 P9157 Tela 6.5 64GB 2GB ram ...,Sobre este item Câmera Tripla 8MP + VGA + V...,7908414420304,Smartphone Multilaser G Max 2 Dourado 64GB 2GB...,americanas,amazon,https://www.americanas.com.br/produto/4782017527,https://www.amazon.com.br/Smartphone-Multilase...
77,Sobre este item Câmera Tripla 8MP + VGA + V...,7908414420304,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,Sobre este item Câmera Tripla 8MP + VGA + V...,7908414420304,Smartphone Multilaser G Max 2 Dourado 64GB 2GB...,amazon,amazon,https://www.amazon.com.br/Smartphone-Multilase...,https://www.amazon.com.br/Smartphone-Multilase...
78,Sobre este item Função de hotspot móvel ...,8906129031876,Smartphone Xiaomi Redmi 9i Sport 64gb / 4gb Ra...,Sistema operacional Nome do sistema operaciona...,8906129031876,Xiaomi Redmi 9i Dual sim 64 gb midnight black ...,amazon,americanas,https://www.amazon.com.br/Smartphone-Xiaomi-Re...,https://www.americanas.com.br/produto/4820569930


## Salvando Pares

In [ ]:
df_pares.to_csv(f"Dados/pares_smartphones.csv", index = False)